In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import cross_val_score

In [2]:
# Read the data file
df = pd.read_pickle('10_first_ppl_100_rots.pkl')

In [3]:
# --------------------------------------
# SECOND STAGE MODEL TRAINING
# --------------------------------------

model_list_2nd_stage = []   # For storing trained models
X_train_list_2nd_stage = [0]*5   # For storing training images, sorted by finger
X_test_list_2nd_stage = [0]*5   # For storing testing images, sorted by finger
y_train_list_2nd_stage = [0]*5   # For storing training identities, sorted by finger
y_test_list_2nd_stage = [0]*5   # For storing testing identities, sorted by finger

for i in range(0,5):
    X = list(df[df.Finger==i]["Image"])
    y = df[df.Finger==i]["Identity"]
    X_train_list_2nd_stage[i], X_test_list_2nd_stage[i], y_train_list_2nd_stage[i], y_test_list_2nd_stage[i] = train_test_split(X,y,test_size=0.2)
    model_list_2nd_stage.append(RandomForestClassifier())
    model_list_2nd_stage[i].fit(X_train_list_2nd_stage[i],y_train_list_2nd_stage[i])
    
# --------------------------------------
# END OF SECOND STAGE MODEL TRAINING
# --------------------------------------

In [4]:
# -------------------------------------
# FIRST STAGE MODEL TRAINING
# -------------------------------------
X_train = [item for sublist in X_train_list_2nd_stage for item in sublist]
X_test = [item for sublist in X_test_list_2nd_stage for item in sublist]
ytr = pd.concat(y_train_list_2nd_stage) # Just for accessing the index of the object
yte = pd.concat(y_test_list_2nd_stage) # Just for accessing the index of the object
y_train = [] # For storing finger
y_test = []# For storing finger

for i in range(len(ytr)):
    j = ytr.index[i]
    y_train.append([j, df["Finger"][j]])   # [index, finger]
for i in range(len(yte)):
    j = yte.index[i]
    y_test.append([j, df["Finger"][j]])   # [index, finger]

y_train_indexfree = [i[1] for i in y_train]

model_finger = RandomForestClassifier()
model_finger.fit(X_train,y_train_indexfree) # Fit the model fo training data
# -------------------------------------
# END OF FIRST STAGE MODEL TRAINING
# -------------------------------------

RandomForestClassifier()

In [5]:
# --------------------------------------
# MODEL IMPLEMENTATION
# --------------------------------------
pred_finger = model_finger.predict(X_test)

X_pred_list = [[]]*5  # Images sorted by predicted finger
y_pred_list = [[]]*5  # True identities sorted by predicted finger

for i in range(len(pred_finger)):
    f = pred_finger[i]
    X_pred_list[f] += [X_test[i]]
    n = y_test[i][0] # Index in DataFrame
    y_pred_list[f] += [df["Identity"][n]]
    
pred_id_list = [[]]*5

for i in range(5):
    pred_id_list[i] = model_list_2nd_stage[i].predict(X_pred_list[i])
# --------------------------------------
# END OF MODEL IMPLEMENTATION
# --------------------------------------

In [6]:
# --------------------------------------
# EVALUATION OF RESULTS
# --------------------------------------
# Calculate accuracy
len_list = [[]]*5
score_list = [0]*5
score_tot = 0
true_id_list = []

for i in range(5):
    len_list[i] = len(pred_id_list[i])

c = 0
for i in range(5):
    siz = len_list[i]
    for j in range(siz):
#         if j<5:
#             print(pred_id_list[i][j])
#             print(df["Identity"][y_test[c][0]])
        true_id_list.append(df["Identity"][y_test[c][0]])
        if pred_id_list[i][j] == df["Identity"][y_test[c][0]]:
            score_list[i] += 1
            score_tot += 1
        c+=1
#         score_list[i] = score_list[i]/siz
    score_tot = score_tot/sum(len_list)

# Print the result
for i in range(len(score_list)):
    print(score_list[i])
print(score_tot)
# --------------------------------------
# END OF EVALUATION OF RESULTS
# --------------------------------------

IndexError: list index out of range

In [ ]:
# Confusion matrix
cm = confusion_matrix(true_id_list, [item for sublist in pred_id_list for item in sublist], normalize='all') # Remove 'normalize="all"' to get absolute numbers
plt.figure()
sn.heatmap(cm, annot=False, cmap='RdPu')
plt.title('Confusion matrix for 2-stage prediction of identity')
plt.xlabel('Predicted')
plt.ylabel('Truth')